### Carregamento de Bibliotecas

In [15]:
import pandas as pd
import polars as pl

### Exercício 1 — Deteção de outliers com IQR

In [147]:
# Leitura Pandas com separador ;
titanic_pandas = pd.read_csv(r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\titanic\train.csv', sep=',', na_values=["N/D", "NA"], encoding="utf-8")

#1.Carregue train.csv para um DataFrame Pandas e para um DataFrame Polars.
df_pandas = pd.DataFrame(titanic_pandas)
#print(df_pandas.head())


#função para IQR pandas
def iqr_pandas(s):
    q1, q3 = s.quantile([0.25, 0.75])
    iqr = q3 - q1
    return q1, q3, iqr

cols = ['Age', 'Fare', 'SibSp']

# 2.Para cada variável, calcule Q1, Q3 e IQR.
print("\nQ1, Q3 e IQR no Pandas")
for col in cols:
    q1, q3, iqr = iqr_pandas(df_pandas[col])
    print(f"Coluna: {col} | Q1: {q1} | Q3: {q3} | IQR: {iqr}")

#3.Defina limites inferior/superior = Q1 − 1,5·IQR e Q3 + 1,5·IQR.
def limites_pandas(s):
    q1, q3, iqr = iqr_pandas(s)
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    return lower_limit, upper_limit 

print("\nLimites Inferior e Superior:")
for col in cols:
    lower_limit, upper_limit = limites_pandas(df_pandas[col])
    print(f"Coluna: {col} | Limite Inferior: {lower_limit:.2f} | Limite Superior: {upper_limit:.2f}")   
    
print("\nContagem de outliers:")    
#4.Conte quantos outliers existem por variável.
for c in cols:
    lower_limit, upper_limit = limites_pandas(df_pandas[c])
    outliers = (df_pandas[c] < lower_limit) | (df_pandas[c] > upper_limit)
    n_outliers = outliers.sum()
    perc= n_outliers / len(df_pandas)
    print(f"Coluna: {c} | Número de Outliers: {n_outliers} | Percentagem: {perc:.2%}")


#5. Compare tempos de execução (use %%timeit).



Q1, Q3 e IQR no Pandas
Coluna: Age | Q1: 20.125 | Q3: 38.0 | IQR: 17.875
Coluna: Fare | Q1: 7.9104 | Q3: 31.0 | IQR: 23.0896
Coluna: SibSp | Q1: 0.0 | Q3: 1.0 | IQR: 1.0

Limites Inferior e Superior:
Coluna: Age | Limite Inferior: -6.69 | Limite Superior: 64.81
Coluna: Fare | Limite Inferior: -26.72 | Limite Superior: 65.63
Coluna: SibSp | Limite Inferior: -1.50 | Limite Superior: 2.50

Contagem de outliers:
Coluna: Age | Número de Outliers: 11 | Percentagem: 1.23%
Coluna: Fare | Número de Outliers: 116 | Percentagem: 13.02%
Coluna: SibSp | Número de Outliers: 46 | Percentagem: 5.16%


In [ ]:
# Leitura Polars com separador ,
titanic_polars = pl.read_csv(r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\titanic\train.csv', separator=',',  null_values=["N/D", "NA"], encoding="utf-8")

#1.Carregue train.csv para um DataFrame Pandas e para um DataFrame Polars.
df_polars = pl.DataFrame(titanic_polars)

#print(df_polars.head())

#função para IQR pandas
def iqr_polars(s):
    q1= df_polars.select(pl.col(s)).quantile(0.25,"nearest").item()
    q3= df_polars.select(pl.col(s)).quantile(0.75,"nearest").item()
    iqr = q3 - q1
    return q1, q3, iqr

cols = ['Age', 'Fare', 'SibSp']

# 2.Para cada variável, calcule Q1, Q3 e IQR.
print("\nQ1, Q3 e IQR no Polars")
for col in cols:
    q1, q3, iqr = iqr_polars(col)
    print(f"Coluna: {col} | Q1: {q1} | Q3: {q3} | IQR: {iqr}")
    
#3.Defina limites inferior/superior = Q1 − 1,5·IQR e Q3 + 1,5·IQR.
def limites_polars(s):
    q1, q3, iqr = iqr_polars(s)
    lower_limit = q1 - 1.5 * iqr
    upper_limit = q3 + 1.5 * iqr
    return lower_limit, upper_limit

print("\nLimites Inferior e Superior:")
for col in cols:
    lower_limit, upper_limit = limites_polars(col)
    print(f"Coluna: {col} | Limite Inferior: {lower_limit:.2f} | Limite Superior: {upper_limit:.2f}")   


#4.Conte quantos outliers existem por variável.
print("\nContagem de outliers:")

for c in cols:
    lower_limit, upper_limit = limites_polars(c)
    outliers = df_polars.filter((pl.col(c) < lower_limit) | (pl.col(c) > upper_limit)) 
    n_outliers = outliers.height
    print(f"Coluna: {c} | Número de Outliers: {n_outliers} ")
    
    

### Exercício 2 — Capping de valores extremos

In [ ]:
#1 - Ler Dataset
df_exam = pd.read_csv(r'C:\Users\HP\Desktop\Formação\Eisnt\UFCD 10808 - Limpeza e transformação de dados em Python\StudentsPerformance.csv', sep=',', na_values=["N/D", "NA"], encoding="utf-8")
#print(df_exam.head())

# 2 - Explorar distribuição inicial
print("\nDistribuição inicial:")
num_cols =['math score', 'reading score', 'writing score']
print(df_exam[num_cols].describe().loc[['mean', 'min', 'max']])

# 3 - Calcular p5/p95 e aplicar capping
print("\nCapping:")
for col in num_cols:
    p5 = df_exam[col].quantile(0.05)
    p95 = df_exam[col].quantile(0.95)
    df_exam[col + "_raw"] = df_exam[col] #guardar os valores originais
    print(f"Coluna: {col} | P5: {p5:.2f} | P95: {p95:.2f}")
    df_exam[col] = df_exam[col].clip(p5, p95)   
    #print(f"Coluna: {col} | Média: {df_exam[col].mean():.2f} | Mínimo: {df_exam[col].min():.2f} | Máximo: {df_exam[col].max():.2f}")
    
# 4 - Avaliar impacto do capping
before = df_exam[[col + "_raw" for col in num_cols]].describe().loc[['mean', 'std']]
after = df_exam[num_cols].describe().loc[['mean', 'std']]
print("\nImpacto do capping:")
print(before.join(after, lsuffix='_before', rsuffix='_after'))

### Exercício 3 — Duplicados & valores nulos

In [ ]:

#1. Verifique duplicados completos 
print("Duplicados completos: ", titanic_pandas.duplicated().sum())

#2.Remova-os
dups = titanic_pandas.duplicated().sum()
print(f"\nNúmero de duplicados: {dups}")
titanic_pandas = titanic_pandas.drop_duplicates()

#3.Impute Age com a mediana por Pclass.
print("\nImputar Age pela mediana estratificada")
age_med = titanic_pandas.groupby('Pclass')['Age'].transform('median')
titanic_pandas['Age'] = titanic_pandas['Age'].fillna(age_med)
print(titanic_pandas['Age'].describe())

#4. Impute Embarked com a moda global.
print(f"\nImputar Embarked pela moda")
most_common = titanic_pandas['Embarked'].mode().iat[0]
titanic_pandas['Embarked'] = titanic_pandas['Embarked'].fillna(most_common)
print(titanic_pandas['Embarked'])

#5. Crie uma flag Cabin_missing (1 = cabine desconhecida).
print(f"\nFlag para cabines desconhecidas")
titanic_pandas['Cabin_missing'] = titanic_pandas['Cabin'].isna().astype(int)
print(titanic_pandas['Cabin_missing'])


### Exercício 4 — Encoding de variáveis categóricas

In [171]:
# 1 - Criar rótulo-alvo
print("\nCriar rótulo-alvo")
df_exam['pass_math'] = (df_exam['math score'] >=60).astype(int)
X_raw = df_exam.drop(columns=['pass_math'])
y = df_exam['pass_math']
cat_cols = ['gender','race/ethnicity','parental level of education','lunch','test preparation course']

#2 - LAbel Encoding
from sklearn.preprocessing import LabelEncoder
X_le = X_raw.copy()
le_dict = {}

for c in cat_cols:
       le = LabelEncoder().fit(X_le[c])
       X_le[c] = le.transform(X_le[c])         
       le_dict[c] = le

#3 - One Hot Encoding
X_oh = pd.get_dummies(X_raw, columns=cat_cols, drop_first=True)
#print(X_oh.head())

#4 - Random Forest + 5-fold CV
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score

clf = RandomForestClassifier(n_estimators=200, random_state=42)
acc_le = cross_val_score(clf, X_le, y, cv=5, scoring='accuracy').mean()
acc_oh = cross_val_score(clf, X_oh, y, cv=5, scoring='accuracy').mean()

print(f"Label Encoding : {acc_le:.2f} \n One-Hot: {acc_oh:.2f}")



Criar rótulo-alvo
Label Encoding : 1.00 
 One-Hot: 1.00


### Exercício 5 — Escalonamento e pipeline